# Loading a semantic map using knowrob

This notebook demonstrates how to load a semantic map into KnowRob

### Importing KnowRob Modules

We will first initialize KnowRob

In [1]:
from enum import Enum
import json
from knowrob import *
    
InitKnowRob()

[15:50:41.623] [info] [KnowRob] static initialization done.


### Setting Up Knowledge Base

Now we set up the knowledge base. See the second data-sources field, to see the path to an expected ontology. Add this ontology before running this cell.

In [2]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
            {"alias": "dul", "uri": "http://www.ontologydesignpatterns.org/ont/dul/DUL.owl"},
			{"alias": "USD", "uri": "https://ease-crc.org/ont/USD.owl"},
            {"alias": "dfl", "uri": "http://www.ease-crc.org/ont/SOMA_DFL.owl"}
		]
	},
	"data-sources": [
		{"path": "owl/USD.owl", "format": "rdf-xml"},
		{"path": "owl/house_example.owl", "format": "rdf-xml"}
	],
	"data-backends": [
    {
      "type": "Redland",
      "name": "redland",
      "read-only": False
    }
	],  
    "reasoner": [
    {
      "type": "SOMADFLReasoner",
      "name": "DFLReasoner",
      "module": "reasoner/dfl_reasoner.py",
      "data-backend": "redland"
    }
    ]
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

def runQuery(queryStr):
    phi = QueryParser.parse(queryStr)
    resultStream = kb.submitQuery(phi, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
    resultQueue = resultStream.createQueue()
    retq = None
    result = resultQueue.pop_front()
    while not result.indicatesEndOfEvaluation():
        if retq is None:
            retq = []
        if isinstance(result, AnswerYes):
            aux = {}
            for substitution in result.substitution():
                variable, term = substitution[1], substitution[2]
                if isinstance(term, Blank):
                    continue
                aux[str(variable)] = str(term)
            retq.append(aux)
        elif isinstance(result, AnswerNo):
            retq = None
            break
        result = resultQueue.pop_front()
    return retq

# Some useful shortcuts in the ontology

class SOMADFL:
    storingTask = "dfl:store.v.wn.possession..place"
    servingFoodTask = 'dfl:serve.v.wn.consumption..concrete'
    eatingPatient = "dfl:eat.v.wn.consumption..food.Patient"
    graspingPatient = "dfl:hold.v.wn.contact..grasp.Theme"
    servingFoodLocation = "http://www.ease-crc.org/ont/SOMA_DFL.owl#serve.v.wn.consumption..concrete.Location"
    breakfastFoodClass = "dfl:breakfast_food.n.wn.food"
    solidFoodClass = "dfl:food.n.wn.food..solid"


[15:50:41.629] [info] Using backend `redland` with type `Redland`.
[15:50:41.631] [info] [redland] using storage of type "memory" with name "knowrob" and options "contexts='yes'".
[15:50:41.631] [info] Using queryable backend with id 'redland'.
[15:50:41.631] [info] Loading ontology at '/home/pomarlan/Documents/knowrob/owl/rdf-schema.xml' with version "Fri Aug 30 12:56:45 2024" and origin "rdf-schema".
[15:50:41.632] [info] Loading ontology at '/home/pomarlan/Documents/knowrob/owl/owl.rdf' with version "Fri Aug 30 12:56:45 2024" and origin "owl".
[15:50:41.633] [info] Loading ontology at '/home/pomarlan/.knowrob/owl/USD.owl' with version "Thu Sep  5 08:48:08 2024" and origin "USD".
[15:50:41.636] [info] Loading ontology at '/home/pomarlan/Documents/knowrob/owl/external/DUL.owl' with version "10-09-2024" and origin "DUL".
[15:50:41.646] [info] Loading ontology at '/home/pomarlan/.knowrob/owl/house_example.owl' with version "Tue Sep 10 14:42:41 2024" and origin "house_example".
[15:50:41

[15:50:58.822] [info] Using goal-driven reasoner with id 'DFLReasoner'.


### Submitting the Competency Questions

Now we we will run the competency questions.

#### CQ1: Which objects do I need for breakfast?

This will return items that are considered breakfast food, and tools that may be needed to serve these. Some scenes may not contain breakfast food, so we will also inquire what food is there in general.


In [3]:
# CQ1: which objects do I need for breakfast? -- reinterpreted as, what objects are subclasses of breakfast food, 
# and what can I use to serve those

# Are there any objects that are instances of breakfast food?
bdgs = runQuery(f"dfl:isInstanceOf(?x, \'{SOMADFL.breakfastFoodClass}\')")
breakfastFoodItems = [x["?x"] for x in (bdgs or [])]

# Are there any objects that contain something that is a breakfast food?

#     First, what kinds of breakfast food are there?
bdgs = runQuery(f"dfl:isSubclassOf(?x, '{SOMADFL.breakfastFoodClass}')")
breakfastFoodClasses = [x["?x"] for x in (bdgs or [])]

#     Second, what containers for these things are around?
containerItems = set()
breakfastFoodClassesPresent = set()
for food in breakfastFoodClasses:
    bdgs = runQuery(f"dfl:hasPart(?x, '{food}')")
    containerItems = containerItems.union([x["?x"] for x in (bdgs or [])])
    if (bdgs is not None) and (0 < len(bdgs)):
        breakfastFoodClassesPresent.add(food)

# What sort of tools are used to serve the breakfast food we found 
#     (either as stand-alone items, or contained somewhere)?
toolItems = set()
for food in set().union(breakfastFoodItems).union(breakfastFoodClassesPresent):
    bdgs = runQuery(f"dfl:useMatch('{SOMADFL.servingFoodTask}', ?x, '{food}')")
    toolItems = toolItems.union([x["?x"] for x in (bdgs or [])])

# Report the results
edibles = containerItems.union(breakfastFoodItems)
print("These things look like breakfast food or contain some breakfast food: %s" % str(sorted(edibles)))
print(" ... and might need these items to serve them: %s" % str(sorted(list(toolItems))))

# Let's also try looking for food in general

bdgs = runQuery(f"dfl:hasDisposition(?x, '{SOMADFL.eatingPatient}')")
foodItems = [x["?x"] for x in (bdgs or [])]

# Are there any objects that contain something that is a breakfast food?

#     First, what kinds of breakfast food are there?
bdgs = runQuery(f"dfl:isSubclassOf(?x, '{SOMADFL.solidFoodClass}')")
foodClasses = [x["?x"] for x in (bdgs or [])]

#     Second, what containers for these things are around?
containerItems = set()
foodClassesPresent = set()
for food in foodClasses:
    bdgs = runQuery(f"dfl:hasPart(?x, '{food}')")
    containerItems = containerItems.union([x["?x"] for x in (bdgs or [])])
    if (bdgs is not None) and (0 < len(bdgs)):
        foodClassesPresent.add(food)

# What sort of tools are used to serve the breakfast food we found 
#     (either as stand-alone items, or contained somewhere)?
toolItems = set()
for food in set().union(foodItems).union(foodClassesPresent):
    bdgs = runQuery(f"dfl:useMatch('{SOMADFL.servingFoodTask}', ?x, '{food}')")
    toolItems = toolItems.union([x["?x"] for x in (bdgs or [])])

# Report the results
edibles = containerItems.union(foodItems)
print("If looking for food in general, these things look like they might fit: %s" % str(sorted(edibles)))
print(" ... and might need these items to serve them: %s" % str(sorted(list(toolItems))))


These things look like breakfast food or contain some breakfast food: ['USD:Egg_6_32']
 ... and might need these items to serve them: ['USD:Fork_6_30']
If looking for food in general, these things look like they might fit: ['USD:Apple_6_41', 'USD:Bread_6_28', 'USD:Bread_6_38', 'USD:Egg_6_32']
 ... and might need these items to serve them: ['USD:Fork_6_30']


#### CQ2 What are the storage locations for our food items?

In [4]:
# CQ2: Where can we find the food?

if 0 < len(edibles):
    locations = {}
    for food in edibles:
        bdgs = runQuery(f"dfl:useMatch('{SOMADFL.storingTask}', ?x, '{food}')")
        locations[food] = [x["?x"] for x in (bdgs or [])]
    
    # Report the results
    print("Places where to look for food:")
    for food in sorted(list(locations.keys())):
        print(f"{food}: {sorted(locations[food])}")
else:
    print("Did not find anything edible in this scene at CQ1 so will skip CQ2 ...")



Places where to look for food:
USD:Apple_6_41: []
USD:Bread_6_28: []
USD:Bread_6_38: []
USD:Egg_6_32: ['USD:Fridge_6_2']


#### CQ3 Where do we expect an item to be?

In [5]:
# CQ3: Where do we expect an item to be? -- note, there is no ranking of locations by likelihood,
# and there may be many plausible locations!

# For example, lets pick a tool from CQ1
if 0 < len(toolItems):
    locations = {}
    for tool in toolItems:
        bdgs = runQuery(f"dfl:useMatch('{SOMADFL.storingTask}', ?x, '{tool}')")
        locations[tool] = [x["?x"] for x in (bdgs or [])]

    # Report results
    print("Places where to look for tools:")
    for tool in sorted(list(locations.keys())):
        print(f"{tool}: {sorted(locations[tool])}")
else:
    print("Did not find any tools at CQ1 so will skip CQ3 ...")



Places where to look for tools:
USD:Fork_6_30: []


#### CQ4 What can I grasp on an object?

In [ ]:
# CQ4: What can I grasp on an object to open it?

# For example, lets pick a tool from CQ1
if 0 < len(toolItems):
    item = list(toolItems)[0]

    # What parts does the object have?
    bdgs = runQuery(f"dul:hasPart('{item}', ?x)")
    parts = [x["?x"] for x in (bdgs or [])]
    parts = set(parts).union([item])
    
    # Which of them are graspable?
    graspables = set()
    for part in parts:
        bdgs = runQuery(f"dfl:hasDisposition('{part}', '{SOMADFL.graspingPatient}')")
        if bdgs is not None:
            graspables.add(part)
    # IF complex queries work (and assuming an object has itself as part, or else a disjunction is needed):
    # bdgs = runQuery(f"dul:hasPart('{item}', ?x), dfl:hasDisposition(?x, 'dfl:hold.v.wn.contact..grasp')")
    # graspables = [x["?x"] for x in bdgs]
    # Report results
    print("Can try to grasp %s by grasping one of %s" % (item, str(sorted(list(graspables)))))
else:
    print("Did not find any tools at CQ1 so will skip CQ4 ...")

#### CQ5 Where should I put the utensils for breakfast?

In [ ]:
# CQ5: Where to place the utensils for breakfast?

bdgs = runQuery(f"dfl:hasDisposition(?x, '{SOMADFL.servingFoodLocation}')")
locations = [x["?x"] for x in (bdgs or [])]

# Report results
print("You could set up the meal at one of these locations: %s" % str(locations))